# Traffic Classification using k Nearest Neighbor

# Background

## Motivation

I am currently working on feature extraction and identification using simple k nearest neighbor clustering algorithms over a data set.

### Preliminary Information

We're using a dataset that has been pre-captured, cleaned and labeled by an ongoing effort found here : https://stratosphereips.org/category/dataset.html

They're currently working to on a machine learning effort for Malware classification, much like the example we're showing here.

The dataset currently contains typical Netflow capture information, including time stamps, durations of the flows, Src/DstIP's, Src/Dst Port, and other flow related information. The key to the provided dataset was that it is labeled. We can then use these labels to start to train a model to identify and classify unlabeled flows as any of the labels provided in the dataset.

In [1]:
# Create Class for formatting of the TimeStamp field contained within the netflow captures

setClass("myPosixCt")
setAs("character", "myPosixCt", function(from) as.POSIXct(from, format = "%Y/%m/%d %H:%M:%OS"))
options(set.seconds="6")

# Read .binetflow file into dataframe

flowdata_csv <- read.csv("capture20110810.binetflow", colClasses = c("myPosixCt", "numeric", "factor", "factor","factor","factor","factor","factor","factor","factor","factor","numeric", "numeric", "numeric", "factor"), strip.white = TRUE, sep = ',')

In [2]:
library(caret)
library(pander)
library(doMC)
library(dplyr)


# Register CPU core count
registerDoMC(cores=3)

# Utility function for use with % frequency tables
frqtab <- function(x, caption) {
    round(100*prop.table(table(x)), 3)
}

# Utility function to round values in a list
# but only if they are numeric

round_numeric <- function(lst, decimals=2) {
    lappy(lst, function(x){
        if (is.numeric(x)) {
            x <- round(x, decimals)
        }
        return(x)
    })
}

# Utility function for model comparison

summod <- function(cm, fit) {
    summ <- list(k = fit$finalModel$k,
                metric = fit$metric,
                value = fit$results[fit$resultes$k == fit$finalModel$k, fit$metric],
                TN = cm$table[1,1], # True negatives
                TP = cm$table[2,2], # True positives
                FN = cm$table[1,2], # False negatives
                FP = cm$table[2,1], # False positives
                acc = cm$overall["Accuracy"], 
                sens = cm$byClass["Sensitivity"],
                spec = cm$byClass["Specificity"],
                PPV = cm$byClass["Positive Predicted Value"],
                NPV = cm$byClass["Negative Prediced Value"])
    round_numeric(summ)
}

# Utility function to normalize the data

normalize <- function(x){
    num <- x - min(x)
    denom <- max(x) - min(x)
    return (num/denom)
}

# Hours Function
hours <- function(h) {
    x <- h * 3600
    return(x)
}

# Minutes Function
mins <- function(m) {
    x <- m * 60
    return(x)
}

# Seconds function
secs <- function(s) {
    x <- s
    return(x)
}

ERROR: Error in library(caret): there is no package called ‘caret’


ERROR: Error in library(pander): there is no package called ‘pander’


ERROR: Error in library(doMC): there is no package called ‘doMC’


ERROR: Error in library(dplyr): there is no package called ‘dplyr’


ERROR: Error in eval(expr, envir, enclos): could not find function "registerDoMC"


In [3]:
# Subset and normalize data

flowdata_csv$Label <- as.character(flowdata_csv$Label)

flowdata_csv <- subset(flowdata_csv, flowdata_csv$StartTime[1] + mins(1) >= flowdata_csv$StartTime)

In [5]:
#Define continuous vars, subset flowdata and save as CSV

contvars <- names(flowdata_csv) %in% c("StartTime", "Proto", "SrcAddr", "Sport", "Dir", "DstAddr", "Dport", "State", "sTos", "dTos")
flowdata_conts <- flowdata_csv[!contvars]

str(flowdata_conts)

# Write out to csv file for persistence

write.csv(flowdata_conts, file='flowdata_conts.csv')

str(flowdata_conts)

'data.frame':	1390379 obs. of  5 variables:
 $ Dur     : num  1.03 1.01 3.06 3.11 3.08 ...
 $ TotPkts : num  4 4 3 3 3 3 4 4 4 5 ...
 $ TotBytes: num  276 276 182 182 182 182 244 252 252 352 ...
 $ SrcBytes: num  156 156 122 122 122 122 124 132 132 208 ...
 $ Label   : chr  "flow=Background-Established-cmpgw-CVUT" "flow=Background-Established-cmpgw-CVUT" "flow=Background-TCP-Attempt" "flow=Background-TCP-Attempt" ...
'data.frame':	1390379 obs. of  5 variables:
 $ Dur     : num  1.03 1.01 3.06 3.11 3.08 ...
 $ TotPkts : num  4 4 3 3 3 3 4 4 4 5 ...
 $ TotBytes: num  276 276 182 182 182 182 244 252 252 352 ...
 $ SrcBytes: num  156 156 122 122 122 122 124 132 132 208 ...
 $ Label   : chr  "flow=Background-Established-cmpgw-CVUT" "flow=Background-Established-cmpgw-CVUT" "flow=Background-TCP-Attempt" "flow=Background-TCP-Attempt" ...


In [6]:
# Normalize the data

cont_vars <- c("Dur", "TotPkts", "TotBytes", "SrcBytes")

flowdata_conts <- flowdata_conts %>% mutate_each_(funs(normalize), vars = cont_vars)

# Clean flowdata_conts, totally hacky but dataframe transforms are crazy fast and scale well

flowdata_conts <- flowdata_conts[!(flowdata_conts$Dur == 0),]
flowdata_conts <- flowdata_conts[!(flowdata_conts$TotPkts == 0),]
flowdata_conts <- flowdata_conts[!(flowdata_conts$TotBytes == 0),]
flowdata_conts <- flowdata_conts[!(flowdata_conts$SrcBytes == 0),]

In [10]:
# Re-factor-fy variable

flowdata_conts$Label <- as.factor(flowdata_conts$Label)

# Set randomization seed

set.seed(1234)

# Break dataset into training and test sets
## split dataset randomly with a 67/33% distribution

ind <- sample(2, nrow(flowdata_conts), replace=TRUE, prob=c(0.67, 0.33))

flowdata_training <- flowdata_conts[ind==1,]
flowdata_test <- flowdata_conts[ind==2,]

#flowdata_training_classes <- flowdata_conts[ind==1,5]
#flowdata_test_classes <- flowdata_conts[ind==2,5]

str(flowdata_training)
str(flowdata_test)

'data.frame':	889063 obs. of  5 variables:
 $ Dur     : num  0.000285 0.00028 0.000849 0.000864 0.00086 ...
 $ TotPkts : num  1.12e-06 1.12e-06 7.44e-07 7.44e-07 7.44e-07 ...
 $ TotBytes: num  8.03e-08 8.03e-08 4.54e-08 4.54e-08 4.54e-08 ...
 $ SrcBytes: num  5.92e-08 5.92e-08 4.63e-08 4.63e-08 4.63e-08 ...
 $ Label   : Factor w/ 106 levels "flow=Background",..: 4 4 23 23 23 4 4 4 24 4 ...
'data.frame':	438530 obs. of  5 variables:
 $ Dur     : num  0.000857 0.091212 0.000256 0.000269 0.000838 ...
 $ TotPkts : num  7.44e-07 2.23e-06 1.12e-06 1.12e-06 7.44e-07 ...
 $ TotBytes: num  4.54e-08 2.60e-07 6.84e-08 6.84e-08 4.61e-08 ...
 $ SrcBytes: num  4.63e-08 1.97e-07 4.71e-08 4.71e-08 4.63e-08 ...
 $ Label   : Factor w/ 106 levels "flow=Background",..: 23 4 4 4 24 4 24 24 24 4 ...


In [11]:
# Display label distribution in datasets

ft_orig <- frqtab(flowdata_conts$Label)
label_freq <- pander(ft_orig, style="rmarkdown", caption="Original Label Frequency (%)")

ft_train <- frqtab(flowdata_training$Label)
ft_test <- frqtab(flowdata_test$Label)
ftcmp_df <- as.data.frame(cbind(ft_orig, ft_train, ft_test))
colnames(ftcmp_df) <- c("Original", "Training Set", "Test Set")
pander(ftcmp_df, style="rmarkdown",
              caption="Comparison of Label frequencies ( in %)")



|  flow=Background  |  flow=Background-ajax.google  |
|:-----------------:|:-----------------------------:|
|       0.694       |             0.046             |

Table: Original Label Frequency (%) (continued below)

 

|  flow=Background-Attempt-cmpgw-CVUT  |
|:------------------------------------:|
|                0.356                 |

Table: Table continues below

 

|  flow=Background-Established-cmpgw-CVUT  |
|:----------------------------------------:|
|                  4.874                   |

Table: Table continues below

 

|  flow=Background-google-analytics1  |  flow=Background-google-analytics10  |
|:-----------------------------------:|:------------------------------------:|
|                0.02                 |                0.038                 |

Table: Table continues below

 

|  flow=Background-google-analytics11  |  flow=Background-google-analytics12  |
|:------------------------------------:|:------------------------------------:|
|                0.0

In [ ]:
# NA omit after cleaning
flowdata_test <- na.omit(flowdata_test)
flowdata_training <- na.omit(flowdata_training)

set.seed(123)

# Create list for seed used with parallelization

seeds <- vector(mode = "list", length = 51)
for (i in 1:50) seeds[[i]] <- sample.int(1000, 22)

# Used for last model
    
seeds[[51]] <- sample.int(1000, 1)

# Define training parameters
    
ctrl <- trainControl(method="repeatedcv", repeats=3, seeds = seeds)

# Run training! LET THE COMPUTER OVERLORD LEARN
    
system.time(
knnFit1 <- train(Label ~., flowdata_training, method="knn",
                trControl = ctrl, tuneLength = 10, preProcess = c("scale", "center", "pca"))
)

# Output stats TODO : convert to markdown for web / slide view

knnFit1

In [ ]:
plot(knnFit1)

In [ ]:
# Run prediction over test dataset
system.time(
knnPredict1 <- predict(knnFit1, newdata = flowdata_test)
)

In [ ]:
# Calculate confusion matric for prediction accuracy
cmat1 <- confusionMatrix(knnPredict1, flowdata_test$Label)

In [ ]:
cmat1